# Credit Card Fraud Detection - Sparkov Dataset

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from utils.offline_preprocessing import convert_bucket_feat, convert_categorical_feat, time_projection, zscore_clipping


In [2]:
df = pd.concat([pd.read_csv('../../../data/raw/fraudTrain.csv'), pd.read_csv('../../../data/raw/fraudTest.csv')], ignore_index=True)
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Pre-Processing

What we don't need to do:
- Deal with missing values, there are none in the dataset
- Deal with duplicates, there are none in the dataset

#### Time feature processing

Generate client age:

In [3]:
timestamp = pd.to_datetime(df['trans_date_trans_time'])
df['age'] = ((timestamp - pd.to_datetime(df['dob']))/np.timedelta64(1, 'Y'))
df['age'] = df['age'].astype(int)
df['age']

0          30
1          40
2          56
3          52
4          32
           ..
1852389    54
1852390    21
1852391    39
1852392    55
1852393    27
Name: age, Length: 1852394, dtype: int64

Generate time between last transaction

In [4]:
def calculate_timedelta(x):
    if x['cc_num'] != x['previous_card']:
        return -1
    else:
        return x['unix_time'] - x['previous_timestamp']

df = df.sort_values(['cc_num', 'unix_time'], ascending=[True, True])

df['trans_timedelta'] = np.nan
df['previous_timestamp'] = df['unix_time'].shift()
df['previous_card'] = df['cc_num'].shift()

df['trans_timedelta'] = df.apply(calculate_timedelta, axis=1)
        
df.drop(['previous_card', 'previous_timestamp'],axis=1,inplace=True)


Convert transaction date

In [5]:
timestamp.dt.month

0           1
1           1
2           1
3           1
4           1
           ..
1852389    12
1852390    12
1852391    12
1852392    12
1852393    12
Name: trans_date_trans_time, Length: 1852394, dtype: int32

In [6]:
df['trans_month_sin'], df['trans_month_cos'] = time_projection(timestamp.dt.month, 30)
df['trans_week_sin'], df['trans_week_cos'] = time_projection(timestamp.dt.day_of_week, 7)
df['trans_hour_sin'], df['trans_hour_cos'] = time_projection(timestamp.dt.hour, 24)

#### Sort dataset back to time

In [7]:
df = df.sort_values('unix_time', ascending=True)
df.reset_index(drop=True)
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-82.048315,0,30,-1.0,0.207912,0.978148,0.781831,0.62349,0.0,1.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.186462,0,40,-1.0,0.207912,0.978148,0.781831,0.62349,0.0,1.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.154481,0,56,-1.0,0.207912,0.978148,0.781831,0.62349,0.0,1.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.561071,0,52,-1.0,0.207912,0.978148,0.781831,0.62349,0.0,1.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-78.632459,0,32,-1.0,0.207912,0.978148,0.781831,0.62349,0.0,1.0


#### Binazrizing gender column:

In [8]:
def gender_binarizer(x):
    if x=='F':
        return 1
    if x=='M':
        return -1
    
df['gender'] = df['gender'].transform(gender_binarizer)
df['gender']

0          1
1          1
2         -1
3         -1
4         -1
          ..
1852389   -1
1852390   -1
1852391    1
1852392   -1
1852393   -1
Name: gender, Length: 1852394, dtype: int64

#### Feature Bucketing

In [9]:
df['lat'] = convert_bucket_feat(df['lat'])
df['long'] = convert_bucket_feat(df['long'])
df['merch_lat'] = convert_bucket_feat(df['merch_lat'])
df['merch_long'] = convert_bucket_feat(df['merch_long'])

/home/nancio/Documents/repos/Thesis/src/data_processing/pre_training/utils/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column.loc[(column >= lower_limit) & (column <= upper_limit)] = (upper_quantile-0.01) * 100
/home/nancio/Documents/repos/Thesis/src/data_processing/pre_training/utils/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column.loc[(column >= lower_limit) & (column <= upper_limit)] = (upper_quantile-0.01) * 100
/home/nancio/Documents/repos/Thesis/src/data_processing/pre_training/utils/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is tr

#### Categorical feature processing

In [10]:
categorical_feat = {
    'lat': 100,
    'long': 100,
    'merch_lat': 100,
    'merch_long': 100,
    'category': 14,
    'cc_num': 999,
    'merchant': 693
}

for key, value in categorical_feat.items():
    df[key] = convert_categorical_feat(df[key], value)

#### Feature Scaling

In [11]:
features = ('amt', 'trans_timedelta', 'trans_month_sin', 'trans_month_cos', 'trans_week_sin', 'trans_week_cos', 'trans_hour_sin', 'trans_hour_cos', 'city_pop')
#, 'trans_timedelta', 
for feat in features:
    df[feat] = zscore_clipping(df[feat])

#### Drop unecesary columns

In [12]:
df.drop(['Unnamed: 0', 'first', 'last', 'job', 'street', 'state', 'trans_num', 'zip', 'city', 'dob'],axis=1,inplace=True)
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
0,2019-01-01 00:00:18,146,585,12,-0.408741,1,29.0,93.0,-0.282429,1325376018,...,25.0,0,30,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
1,2019-01-01 00:00:44,51,105,2,0.233378,1,62.0,50.0,-0.293527,1325376044,...,79.0,0,40,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
2,2019-01-01 00:00:51,860,366,7,0.942184,-1,69.0,95.0,-0.280243,1325376051,...,73.0,0,56,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
3,2019-01-01 00:01:16,696,39,1,-0.157381,-1,37.0,66.0,-0.287590,1325376076,...,72.0,0,52,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
4,2019-01-01 00:03:06,195,521,11,-0.176470,-1,62.0,79.0,-0.293693,1325376186,...,93.0,0,32,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818


#### Split the dataset to their original format

Split dataset by the predfined time in kaggle:

In [13]:
train = df[df['trans_date_trans_time'] < '2020-06-21 12:14:25']
test = df[df['trans_date_trans_time'] >= '2020-06-21 12:14:25']

Verify if split was done successfully:

In [14]:
train.tail()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
1296670,2020-06-21 12:12:08,336,308,7,-0.342243,-1,35.0,95.0,-0.293165,1371816728,...,73.0,0,58,-0.256158,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296671,2020-06-21 12:12:19,800,456,8,-0.115310,-1,62.0,84.0,-0.293689,1371816739,...,76.0,0,40,-0.455223,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296672,2020-06-21 12:12:32,194,408,8,0.225215,-1,40.0,66.0,-0.291039,1371816752,...,96.0,0,52,0.021323,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296673,2020-06-21 12:13:36,261,331,8,0.030369,-1,58.0,82.0,-0.290286,1371816816,...,48.0,0,39,1.419540,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296674,2020-06-21 12:13:37,333,431,8,-0.412948,-1,60.0,95.0,-0.293298,1371816817,...,72.0,0,24,-0.634966,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575


In [15]:
test.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
1296675,2020-06-21 12:14:25,288,442,9,-0.421990,-1,63.0,72.0,0.812151,1371816865,...,93.0,0,52,1.426876,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296676,2020-06-21 12:14:33,157,392,9,-0.252575,1,62.0,100.0,-0.293019,1371816873,...,82.0,0,30,-0.102960,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296677,2020-06-21 12:14:53,95,468,10,-0.180740,1,62.0,67.0,-0.179602,1371816893,...,62.0,0,49,-0.297127,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296678,2020-06-21 12:15:15,460,504,11,-0.062878,-1,37.0,72.0,-0.112365,1371816915,...,67.0,0,32,0.987213,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575
1296679,2020-06-21 12:15:17,165,649,14,-0.419918,-1,47.0,42.0,-0.290286,1371816917,...,32.0,0,65,-0.406467,0.859281,0.428411,-1.032251,0.671264,0.197699,-1.392575


#### Save to .csv files

In [16]:
train.to_csv("../../../data/modified/dataframes/modified_fraudTrain.csv")
test.to_csv("../../../data/modified/dataframes/modified_fraudTest.csv")